In [2]:
import os
import re
import math
import cv2
import pandas as pd
import numpy as np

from tqdm import tqdm
from glob import glob
from tools.settings import *
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers import Dense, Dropout
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical
from tools.train_val_test_spliter import split

In [112]:
# To split dataset if already splits folder already exits no need to run it
split()

Splitting the ginen dataset into Train Test=0.4 Validation=0.1
Done


In [3]:
train = pd. read_csv(os.path.join(dataset_path, "train.csv"))
test = pd. read_csv(os.path.join(dataset_path, "test.csv"))
val = pd. read_csv(os.path.join(dataset_path, "val.csv"))

In [3]:
train.head()

,Video_url,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch


In [4]:
test.head()

,Video_url,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [11]:
val.head()

,Video_url,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [5]:
print(train.shape)
print(test.shape)
print(val.shape)

(111, 2)


In [13]:
def convert_to_frame(data, folder_name):
    '''
    Generated filenames format dataset_path/folder_name/video_name_frame{number}_action.jpg
    '''
    directory = os.path.join(dataset_path, folder_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    for i in tqdm(range(data.shape[0])):
        video_file = data['Video_url'][i]
        action = data['action'][i]
        video_name_list = video_file.split('/')[-1].split('.')
        video_name_list = video_name_list[:-1]
        video_name = ""
        for n in video_name_list:
            video_name += n
        # capturing the video from the given path
        capture = cv2.VideoCapture(video_file) 
        #frame rate
        frame_rate = capture.get(5)
        count = 0
        while(capture.isOpened()):
            #current frame number
            frame_id = capture.get(1) 
            read_correctly, frame = capture.read()
            if not read_correctly:
                break
            if (frame_id % math.floor(frame_rate) == 0):
                # storing the frames in a new folder named train_1
                filename = directory + "/" + video_name + "_frame{}_".format(count) + action +".jpg"
                count += 1
                cv2.imwrite(filename, frame)
        capture.release()
    print("Successfully Converted")

In [89]:
convert_to_frame(train, train_frames_path_name)

100%|██████████| 143/143 [00:50<00:00,  2.84it/s]

Successfully Converted


In [15]:
convert_to_frame(val, val_frames_path_name)

100%|██████████| 27/27 [00:09<00:00,  2.89it/s]

Successfully Converted


In [17]:
def create_paths_csv(directory, file_name):
    images = os.listdir(directory)
    images_path_list = []
    images_action_list = [] 
    for image in images:
        images_path_list.append(directory + image)
        images_action_list.append(image.split('.')[0].split('_')[-1])
    df = pd.DataFrame()
    df['image'] = images_path_list
    df['action'] = images_action_list
    print(os.path.join(dataset_path, file_name+'.csv'))
    df.to_csv(os.path.join(dataset_path, file_name+'.csv'), index=False)

In [121]:
create_paths_csv(train_frames_path, train_frames_path_name)

/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-time-action-recognition-from-video-footage/dataset/train_frames.csv


In [18]:
create_paths_csv(val_frames_path, val_frames_path_name)

/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-time-action-recognition-from-video-footage/dataset/val_frames.csv


In [36]:
train_image = pd.read_csv(os.path.join(dataset_path, 'train_frames.csv'))
train_image.head()

,image,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [3]:
print(train_image.shape)

(620, 2)


In [4]:
val_image = pd.read_csv(os.path.join(dataset_path, 'val_frames.csv'))
val_image.head()

,image,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [5]:
print(val_image.shape)

(115, 2)


In [33]:
action_values = list(train_image['action'].unique())
action_values

['punch', 'slap', 'kick']

In [37]:
def create_class_columns(df):
    for value in action_values:
        df[value] = np.where(df['action'].str.contains(value), 1, 0)
    df.drop('action', axis='columns', inplace=True)

In [38]:
# train_image = train_image.replace(action_nums)
create_class_columns(train_image)
train_image.head()

,image,punch,slap,kick
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,1,0,0
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0,1,0
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0,1,0
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0,1,0
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0,1,0


In [39]:
# val_image = val_image.replace(action_nums)
create_class_columns(val_image)
val_image.head()

,image,punch,slap,kick
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0,1,0
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0,1,0
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0,0,1
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0,1,0
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0,1,0


In [40]:
def convert_to_array_and_split(image_data):
    image_value = []
    for i in tqdm(range(image_data.shape[0])):
        img = image.load_img(image_data['image'][i], target_size=(224,224,3))
        img = image.img_to_array(img)
        # normalizing the pixel value
        img = img / 255
        image_value.append(img)

    X = np.array(image_value)
    y = image_data
    y.drop('image', axis='columns', inplace=True)
    return X, y

In [41]:
X_train, y_train = convert_to_array_and_split(train_image)
print(X_train.shape)

100%|██████████| 620/620 [00:06<00:00, 88.84it/s] 


(620, 224, 224, 3)


In [42]:
X_val, y_val = convert_to_array_and_split(val_image)
print(X_val.shape)

100%|██████████| 115/115 [00:01<00:00, 89.14it/s]


(115, 224, 224, 3)


In [43]:
y_train.head()

,punch,slap,kick
0,1,0,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


In [44]:
y_val.head()

,punch,slap,kick
0,0,1,0
1,0,1,0
2,0,0,1
3,0,1,0
4,0,1,0


In [6]:
'''This model was trained on a dataset that has 1,000 classes. 
include_top = False will remove the last layer of this model so that we can tune it as per our need.
'''
base_model = VGG16(weights='imagenet', include_top=False)

In [46]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(620, 7, 7, 512)

In [47]:
X_val = base_model.predict(X_val)
X_val.shape

(115, 7, 7, 512)

In [48]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(620, 7*7*512)
X_val = X_val.reshape(115, 7*7*512)

In [49]:
# normalizing the pixel values
max_pixel = X_train.max()
X_train = X_train / max_pixel
X_val = X_val / max_pixel
print(X_train.shape)
print(X_val.shape)

(620, 25088)
(115, 25088)


In [22]:
# y_train.value_counts()
# y_train = to_categorical(y_val, 3)
# y_val = to_categorical(y_val, 3)
# print(y_train)

IndexError: index 3 is out of bounds for axis 1 with size 3

In [7]:
# The input shape will be 25,088
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

In [51]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_weight = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [52]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[mcp_weight], batch_size=128)

Epoch 1/50
5/5 [==============================] - 3s 321ms/step - loss: 1.1389 - accuracy: 0.3393 - val_loss: 1.0301 - val_accuracy: 0.4174
Epoch 2/50
5/5 [==============================] - 1s 218ms/step - loss: 1.1504 - accuracy: 0.4171 - val_loss: 1.0465 - val_accuracy: 0.5478
Epoch 3/50
5/5 [==============================] - 1s 213ms/step - loss: 1.1016 - accuracy: 0.4032 - val_loss: 0.9591 - val_accuracy: 0.6000
Epoch 4/50
5/5 [==============================] - 1s 215ms/step - loss: 1.0401 - accuracy: 0.4225 - val_loss: 0.8997 - val_accuracy: 0.7130
Epoch 5/50
5/5 [==============================] - 1s 241ms/step - loss: 0.9412 - accuracy: 0.5252 - val_loss: 0.8153 - val_accuracy: 0.6000
Epoch 6/50
5/5 [==============================] - 1s 270ms/step - loss: 0.8623 - accuracy: 0.5709 - val_loss: 0.7721 - val_accuracy: 0.5826
Epoch 7/50
5/5 [==============================] - 1s 246ms/step - loss: 0.7634 - accuracy: 0.5906 - val_loss: 0.7163 - val_accuracy: 0.6435
Epoch 8/50
5/5 [====

In [8]:
model.load_weights("weight.hdf5")
# model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [18]:
from scipy import stats as s
predict = []
actual = []
if not os.path.exists(test_frames_path):
    os.makedirs(test_frames_path)

for i in tqdm(range(test.shape[0])):
    video_file = test['Video_url'][i]
    action = test['action'][i]
    video_name_list = video_file.split('/')[-1].split('.')
    video_name_list = video_name_list[:-1]
    video_name = ""
    for n in video_name_list:
        video_name += n
    # capturing the video from the given path
    capture = cv2.VideoCapture(video_file) 
    #frame rate
    frame_rate = capture.get(5)
    count = 0
    files = glob(test_frames_path + '/*')
    #removing all files from folder
    for f in files:
        os.remove(f)
    while(capture.isOpened()):
        #current frame number
        frame_id = capture.get(1) 
        read_correctly, frame = capture.read()
        if not read_correctly:
            break
        if (frame_id % math.floor(frame_rate) == 0):
            # storing the frames in a new folder named train_1
            filename = test_frames_path + "/" + video_name + "_frame{}_".format(count) + action +".jpg"
            count += 1
            cv2.imwrite(filename, frame)
    capture.release()
    
    # reading all the frames from temp folder
    images = glob(test_frames_path + '/*.jpg')
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img / 255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
    prediction = np.argmax(model.predict(prediction_images), axis=-1)
    print(prediction)
    print(s.mode(prediction)[0][0])
    break
    #need to see what happend
    # appending the mode of predictions in predict list to assign the tag to the video
#     predict.append(y.columns.values[s.mode(prediction)[0][0]])
#     # appending the actual tag of the video
#     actual.append(videoFile.split('/')[1].split('_')[1])

  0%|          | 0/111 [00:01<?, ?it/s]

[0 0 0 0]
0
